In [26]:
from terra_sdk.client.lcd import LCDClient
from terra_sdk.key.mnemonic import MnemonicKey
from terra_sdk.client.lcd.api.tx import CreateTxOptions
from terra_sdk.core import Coins, Coin
from terra_sdk.core.bank import MsgSend
from terra_sdk.core.fee import Fee
from terra_sdk.core import Coins
import requests
import json
import math

In [42]:
# SEED PHRASE OF SEND WALLET, RANDOM ONE SHOWN HERE
MNEMONIC = "pioneer stick lobster improve observe ritual wood gym planet purity chalk craft unable alley amazing moral cry ankle element image army clap renew river"

# Gets prices and sets up the LCDClient
prices = requests.get("https://fcd.terra.dev/v1/txs/gas_prices")
terra = LCDClient(
    url="https://rebel1.grouptwo.org",
    chain_id="rebel-2",
    gas_prices=Coins(prices.json()),
    gas_adjustment="3.0"
)


In [4]:
# Load lost fees json
with open('lost_fees.json') as user_file:
  file_contents = user_file.read()

parsed_json = json.loads(file_contents)
print(parsed_json)

{'wallets': {'terra1lgdpa7xl7n5k9wg65chldpkc06j2p5kg2cgf8w': {'tx': 8, 'uluna': 52800000, 'uusd': 0.06386}, 'terra1e8zul28wl8nzj9dvnw82lww7kkw3ypt04xgf2p': {'tx': 4, 'uluna': 41017354.180700004, 'uusd': 0.031888}, 'terra1sm88h365jjpu2ennlpp0sw3l86p6qt3g57zx5j': {'tx': 598, 'uluna': 15264873.1330502}, 'terra19g9v7wzfjgj5tvxmx734l75va2v3z6sakce0gn': {'tx': 4, 'uluna': 14073826.556725}, 'terra1el3skj5qn4qvherz0kpxdydn2vrad7dzrp2k0r': {'tx': 8, 'uluna': 13856538.703797, 'uusd': 0.06362}, 'terra1t6p6zgqqyzzknkvfq3uvq46tvwjj0cwck9369m': {'tx': 2, 'uluna': 10805035.277849, 'uusd': 0.015905}, 'terra10r2ppskhc2fmfq6ep3vgpepnss38tgly48ndul': {'tx': 40, 'uluna': 7938462.513536, 'uusd': 0.31800599999999996}, 'terra1e3nn3dmjc5ngerkvkrjh9jpe23nv4c5hyta5n2': {'tx': 10, 'uluna': 6368462.906401, 'uusd': 0.079525}, 'terra1ma29ptjw5g27k89529s9y7jj2apa08ft8xupz8': {'tx': 148, 'uluna': 6203050.943960001, 'uusd': 1.1613229999999992, 'ukrw': 18.02544}, 'terra1nj7ru7yt6n5skvjv29gjyk9ke2xy3grfw3hqpe': {'tx': 8

In [17]:
print(len(parsed_json['wallets']))
print(parsed_json['wallets']['terra1lgdpa7xl7n5k9wg65chldpkc06j2p5kg2cgf8w'])

2761
{'tx': 8, 'uluna': 52800000, 'uusd': 0.06386}


In [25]:
count = 0
for wallet in parsed_json['wallets']:
    print(wallet)
    if parsed_json['wallets'][wallet].get('uluna') != None:
        if parsed_json['wallets'][wallet]['uluna'] > 5000000 or parsed_json['wallets'][wallet]['uluna'] < 10:
            print("NOT DISTRIBUTED",parsed_json['wallets'][wallet]['uluna'])
        else:
            print("WILL DISTRIBUTE",parsed_json['wallets'][wallet]['uluna'])
            count = count + 1
print(count)

terra1lgdpa7xl7n5k9wg65chldpkc06j2p5kg2cgf8w
NOT DISTRIBUTED 52800000
terra1e8zul28wl8nzj9dvnw82lww7kkw3ypt04xgf2p
NOT DISTRIBUTED 41017354.180700004
terra1sm88h365jjpu2ennlpp0sw3l86p6qt3g57zx5j
NOT DISTRIBUTED 15264873.1330502
terra19g9v7wzfjgj5tvxmx734l75va2v3z6sakce0gn
NOT DISTRIBUTED 14073826.556725
terra1el3skj5qn4qvherz0kpxdydn2vrad7dzrp2k0r
NOT DISTRIBUTED 13856538.703797
terra1t6p6zgqqyzzknkvfq3uvq46tvwjj0cwck9369m
NOT DISTRIBUTED 10805035.277849
terra10r2ppskhc2fmfq6ep3vgpepnss38tgly48ndul
NOT DISTRIBUTED 7938462.513536
terra1e3nn3dmjc5ngerkvkrjh9jpe23nv4c5hyta5n2
NOT DISTRIBUTED 6368462.906401
terra1ma29ptjw5g27k89529s9y7jj2apa08ft8xupz8
NOT DISTRIBUTED 6203050.943960001
terra1nj7ru7yt6n5skvjv29gjyk9ke2xy3grfw3hqpe
WILL DISTRIBUTE 4848000
terra1fw58wa2fcmd2hv09c9xk3gkzunegwk7tzs95qt
WILL DISTRIBUTE 4779866.543296
terra1hha3ygfmm3z2vnsx3ww0jad67a37nynsmxffn6
WILL DISTRIBUTE 4638894.104366999
terra184wxn27fw370hxewlh2fj8za36q5676x0xq0vp
WILL DISTRIBUTE 3725574.271547
terra1puk3

In [53]:
# Creates wallet to initiate transactions
results = {}
mk = MnemonicKey(mnemonic=MNEMONIC)
sendwallet = terra.wallet(mk)

for wallet in parsed_json['wallets']:
    #print(wallet)
    if parsed_json['wallets'][wallet].get('uluna') != None:
        if parsed_json['wallets'][wallet]['uluna'] > 5000000 or parsed_json['wallets'][wallet]['uluna'] < 10:
            print("NOT DISTRIBUTED",parsed_json['wallets'][wallet]['uluna'])
        else:
            print("WILL DISTRIBUTE",parsed_json['wallets'][wallet]['uluna'])
            
            amountuluna = int(parsed_json['wallets'][wallet]['uluna']*1000000)
            taxes = math.ceil(amountuluna * 0.002)
            total = math.ceil(taxes + (200000 * 28.325))

            # Creates transaction
            tx_options = CreateTxOptions(
                msgs=[
                    MsgSend(
                        from_address="terra14j5f8kxjdgvzkf7ne5g9lldl8zx4vjfkzuzk49",
                        to_address=wallet,
                        amount=Coins([Coin("uluna", (amountuluna-total))])
                    )
                ],
                fee=Fee(200000, str(total)+"uluna")
            )
            tx = sendwallet.create_and_sign_tx(options=tx_options)

            # Broadcasts the transactions and prints results
            result = terra.tx.broadcast(tx)
            print(result)
            print(result.txhash)
            print(total)
            results[wallet] = result.txhash

NOT DISTRIBUTED 52800000
NOT DISTRIBUTED 41017354.180700004
NOT DISTRIBUTED 15264873.1330502
NOT DISTRIBUTED 14073826.556725
NOT DISTRIBUTED 13856538.703797
NOT DISTRIBUTED 10805035.277849
NOT DISTRIBUTED 7938462.513536
NOT DISTRIBUTED 6368462.906401
NOT DISTRIBUTED 6203050.943960001
NOT DISTRIBUTED 4848000
NOT DISTRIBUTED 4779866.543296
NOT DISTRIBUTED 4638894.104366999
NOT DISTRIBUTED 3725574.271547
NOT DISTRIBUTED 3576000
NOT DISTRIBUTED 3111714.756
NOT DISTRIBUTED 3000000
NOT DISTRIBUTED 2453437.207678
NOT DISTRIBUTED 2423496
NOT DISTRIBUTED 2390031.735022
NOT DISTRIBUTED 1912849.799772
NOT DISTRIBUTED 1815733.439924
NOT DISTRIBUTED 1814058.302078
NOT DISTRIBUTED 1500000
NOT DISTRIBUTED 1472756.856
NOT DISTRIBUTED 1471035.649208
NOT DISTRIBUTED 1440000.006118
NOT DISTRIBUTED 1410415.683348
NOT DISTRIBUTED 1380120
NOT DISTRIBUTED 1256147.179012
NOT DISTRIBUTED 1224000
NOT DISTRIBUTED 1213200
NOT DISTRIBUTED 1188006
NOT DISTRIBUTED 1167746.73022
NOT DISTRIBUTED 1098720.984
NOT DISTRI

BlockTxBroadcastResult(height=12475125, txhash='69009831C34D3EC603DFBF8828A5A87E3FBC77FC7C93BE52CD9820502D96486C', raw_log='[{"events":[{"type":"coin_received","attributes":[{"key":"receiver","value":"terra1yj7x2jhv7hjqsn2px39fsugapl6syr09pdd3ks"},{"key":"amount","value":"5113400uluna"}]},{"type":"coin_spent","attributes":[{"key":"spender","value":"terra14j5f8kxjdgvzkf7ne5g9lldl8zx4vjfkzuzk49"},{"key":"amount","value":"5113400uluna"}]},{"type":"message","attributes":[{"key":"action","value":"/cosmos.bank.v1beta1.MsgSend"},{"key":"sender","value":"terra14j5f8kxjdgvzkf7ne5g9lldl8zx4vjfkzuzk49"},{"key":"module","value":"bank"}]},{"type":"transfer","attributes":[{"key":"recipient","value":"terra1yj7x2jhv7hjqsn2px39fsugapl6syr09pdd3ks"},{"key":"sender","value":"terra14j5f8kxjdgvzkf7ne5g9lldl8zx4vjfkzuzk49"},{"key":"amount","value":"5113400uluna"}]}]}]', gas_wanted=200000, gas_used=83532, logs=[TxLog(msg_index=0, log='', events=[{'type': 'coin_received', 'attributes': [{'key': 'receiver', 'v

BlockTxBroadcastResult(height=12475129, txhash='037E9F68538686C3E4F2C9A783601D5F9F16EB295FC0486938797BE9FD621C0A', raw_log='[{"events":[{"type":"coin_received","attributes":[{"key":"receiver","value":"terra1f2q2rjv4tk2sjm7f209t0ljflamy7pdxps67l5"},{"key":"amount","value":"5065495uluna"}]},{"type":"coin_spent","attributes":[{"key":"spender","value":"terra14j5f8kxjdgvzkf7ne5g9lldl8zx4vjfkzuzk49"},{"key":"amount","value":"5065495uluna"}]},{"type":"message","attributes":[{"key":"action","value":"/cosmos.bank.v1beta1.MsgSend"},{"key":"sender","value":"terra14j5f8kxjdgvzkf7ne5g9lldl8zx4vjfkzuzk49"},{"key":"module","value":"bank"}]},{"type":"transfer","attributes":[{"key":"recipient","value":"terra1f2q2rjv4tk2sjm7f209t0ljflamy7pdxps67l5"},{"key":"sender","value":"terra14j5f8kxjdgvzkf7ne5g9lldl8zx4vjfkzuzk49"},{"key":"amount","value":"5065495uluna"}]}]}]', gas_wanted=200000, gas_used=83532, logs=[TxLog(msg_index=0, log='', events=[{'type': 'coin_received', 'attributes': [{'key': 'receiver', 'v

BlockTxBroadcastResult(height=12475133, txhash='AE9616CA49FC9C160ADAC7BC4610D63AD71A63F3C1CFEDF59A2C90685810CDE3', raw_log='[{"events":[{"type":"coin_received","attributes":[{"key":"receiver","value":"terra1xkp5ta64uz5us2xjfx3d6dc5ckcz4u7a2m4kn4"},{"key":"amount","value":"4550528uluna"}]},{"type":"coin_spent","attributes":[{"key":"spender","value":"terra14j5f8kxjdgvzkf7ne5g9lldl8zx4vjfkzuzk49"},{"key":"amount","value":"4550528uluna"}]},{"type":"message","attributes":[{"key":"action","value":"/cosmos.bank.v1beta1.MsgSend"},{"key":"sender","value":"terra14j5f8kxjdgvzkf7ne5g9lldl8zx4vjfkzuzk49"},{"key":"module","value":"bank"}]},{"type":"transfer","attributes":[{"key":"recipient","value":"terra1xkp5ta64uz5us2xjfx3d6dc5ckcz4u7a2m4kn4"},{"key":"sender","value":"terra14j5f8kxjdgvzkf7ne5g9lldl8zx4vjfkzuzk49"},{"key":"amount","value":"4550528uluna"}]}]}]', gas_wanted=200000, gas_used=83502, logs=[TxLog(msg_index=0, log='', events=[{'type': 'coin_received', 'attributes': [{'key': 'receiver', 'v

In [52]:
print(results)

{'terra1yj7x2jhv7hjqsn2px39fsugapl6syr09pdd3ks': '6770F45CD00D6256CF0C40691ED4FAE1E9C7ED60687D9FF961C06CA622486489', 'terra1cyyewdcy8wxgnvdgs79zgrck2d7evxmwruuqpq': '4880D0767B479180A9F55D81E99BE8D530320E80FD6B1A2C7F63523F7DCE5063', 'terra18gr53n98swtdz4puj4akfkzvn0z028zedcdxk5': 'ACAC96AD58BD54015F309A31398A4280B800F4F63EE89CB876691F47A07B0109', 'terra1lh0fvg2vh7r2qghv835u0uen7knnx2njaze9n2': '63595B5CC02B8B90BA31FEE7F0EF2EC90239B019AE02CD76BCB8DB5570691CF4', 'terra1f2q2rjv4tk2sjm7f209t0ljflamy7pdxps67l5': 'F54DB585E0D1C2065D6184ED913F8BB5829C6ACC1E52BCF387C9E5FE3807EBA3', 'terra1pez2fg04t3ysnjzhr2u7pgl97ftujckt099ctk': 'ABBE3939AD8389727F530C00A7E5058D41D599080D4EFDAD8F63850475DA9F5E', 'terra13e9yss46s6yyhlggwxl6c9javst2xsyshqpely': 'DAB3573FA79A1D580307260166E7C226B62C64197A501DE23B72664F62EB0D2C', 'terra1aujc054avxera75c300y0cf0raxnur5e3qmclv': '742BCDE44BFA71EA05B0BE28C44B5F4B0134CF8327FFCC88E9BDB5E739DE69DF', 'terra1xkp5ta64uz5us2xjfx3d6dc5ckcz4u7a2m4kn4': 'C6672029247414982A31FF